In [ ]:
import random
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable


class Encoder(nn.Module):

    def __init__(self, vocab_size, embedding_size, hidden_size, n_layers=1, dropout=0.5):
        """
        Bidirectional LSTM encoder; aggregate the outputs of two directions by summation
        :param int vocab_size: size of the source language vocabulary
        :param int embedding_size: size of embedded
        :param int hidden_size: size of hidden states
        :param int n_layers: number of layers
        :param float dropout: dropout rate
        """
        super(Encoder, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, dropout=dropout, bidirectional=True)

    def forward(self, input_sentence, hidden=None):
        """
        forward propagation
        :param torch.Tensor input_sentence: source sentence; shape of (T, B)
        :param tuple hidden: previous hidden states, (h, c)
        :return: outputs sequence, the last hidden states include h and c
        """
        embedded = self.embedding(input_sentence)  # (T, B, E)
        outputs, hidden = self.lstm(embedded, hidden)
        # aggregate hidden states of two directions by sum
        outputs = (outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:])  # (T, B, H)
        return outputs, hidden


class Attention(nn.Module):

    def __init__(self, hidden_size):
        """
        attention weights; two-layer MLP as alignment model
        :param int hidden_size: the hidden state size of encoder
        """
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn1 = nn.Linear(2*hidden_size, hidden_size)  # first fc layer
        self.attn2 = nn.Linear(hidden_size, 1)  # second fc layer

    def forward(self, encoder_outputs, hidden):
        """
        calculate attention score
        :param torch.Tensor encoder_outputs: sequence of outputs from encoder
        :param torch.Tensor hidden: the previous hidden h of decoder, shape of (1, B, H)
        :return: attention score
        """
        length = encoder_outputs.size(0)
        encoder_outputs = encoder_outputs.transpose(0, 1)  # (B, T, H)
        hidden = hidden.repeat(length, 1, 1).transpose(0, 1)  # (B, T, H)
        # calculate energies
        energies = self.attn1(torch.cat([encoder_outputs, hidden], dim=2))
        energies = F.relu(energies)  # (B, T, H)
        energies = self.attn2(energies)  # (B, T, 1)
        # calculate attention weights
        attention = F.softmax(energies.squeeze(2), dim=1).unsqueeze(1)  # (B, 1, T)
        return attention


class Decoder(nn.Module):

    def __init__(self, vocab_size, embedding_size, hidden_size, n_layers=1, dropout=0.5):
        """
        :param int vocab_size: size of the target language vocabulary
        :param int embedding_size: size of embedded
        :param int hidden_size: size of hidden states
        :param int n_layers: number of layers
        :param float dropout: dropout rate
        """
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size + hidden_size, hidden_size, n_layers, dropout=dropout)
        self.attention = Attention(hidden_size)  # encoder and decoder will use same hidden size for simplicity
        self.out = nn.Linear(2*hidden_size, vocab_size)

    def forward(self, input_word, hidden, encoder_outputs):
        """
        forward propagation
        :param torch.Tensor input_word: the input word tensor, shape of (B, )
        :param tuple hidden: previous hidden states
        :param torch.Tensor encoder_outputs: output from encoder
        :return: log likelihood of output words; final hidden states
        """
        # get input word embeddings
        embedded = self.embedding(input_word)  # (B, E)
        embedded = embedded.unsqueeze(0)  # (1, B, E)
        # calculate context
        attention = self.attention(encoder_outputs, hidden[0][-1:])  # hidden h uses the last layer
        context = attention.bmm(encoder_outputs.transpose(0, 1))  # (B, 1, H)
        context = context.transpose(0, 1)  # (1, B, H)
        # concatenate LSTM input
        lstm_input = torch.cat([embedded, context], dim=2)  # (1, B, E+H)
        output, hidden = self.lstm(lstm_input, hidden)
        # concatenate final layer input
        context = context.squeeze(0)  # (B, H)
        output = output.squeeze(0)  # (B, H)
        final_input = torch.cat([output, context], dim=1)  # (B, 2H)
        # final output layer
        output = self.out(final_input)  # (B, N)
        output = F.log_softmax(output, dim=1)
        return output, hidden


class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder):
        """
        the final Seq2Seq model with attention
        :param Encoder encoder: the Encoder class
        :param Decoder decoder: the Decoder class
        """
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing=0.5):
        """
        forward propagation
        :param torch.Tensor source: the source sentences, shape of (T', B)
        :param torch.Tensor target: the target sentences, shape of (T, B)
        :param float teacher_forcing: the teacher forcing ratio
        :return: translated sentences
        """
        # initialize outputs
        target_len = target.size(0)
        batch_size = source.size(1)
        vocab_size = self.decoder.vocab_size
        outputs = Variable(torch.zeros(target_len, batch_size, vocab_size))  # (T, B, N)

        # encoding source sentences
        encoder_outputs, hidden = self.encoder(source)
        # initialize decoder as the final hidden states of encoder
        hidden = (hidden[0][:self.decoder.n_layers], hidden[1][:self.decoder.n_layers])

        # decoding word by word
        output = Variable(target.data[0, :])  # first output <SOS>, shape of (B,)
        for t in range(1, target_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output  # fill in outputs
            top1 = output.max(1)[1]  # the word index with the largest log likelihood
            # teacher forcing
            is_teacher = random.random() < teacher_forcing
            output = Variable(target.data[t] if is_teacher else top1)
        return outputs

    def translate(self, source, max_length):
        """
        translate one sentence
        :param torch.Tensor source: the source sentence, shape of (T', 1)
        :param int max_length: max translation length
        :return: translated sentence
        """
        # encoding source sentences
        encoder_outputs, hidden = self.encoder(source)
        # initialize decoder as the final hidden states of encoder
        hidden = (hidden[0][:self.decoder.n_layers], hidden[1][:self.decoder.n_layers])
        outputs = []
        # decoding word by word
        output = torch.LongTensor([2])  # first output <SOS>, shape of (1,)
        for t in range(1, max_length+1):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            top1 = output.max(1)[1]  # the word index with the largest log likelihood
            outputs.append(top1.item())  # fill in outputs
            # break if <EOS>
            if top1 == 3:
                break
            output = top1
        return outputs


if __name__ == "__main__":
    test_encoder = Encoder(20, 10, 5)
    test_decoder = Decoder(20, 10, 5)
    model = Seq2Seq(test_encoder, test_decoder)

    src = torch.randint(high=20, size=(20, 8))
    trg = torch.randint(high=20, size=(10, 8))

    res = model(src, trg)
    print(res.shape)

    test_sen = torch.randint(high=20, size=(20, 1))
    res = model.translate(test_sen, 10)
    print(res)

torch.Size([10, 8, 20])
[18, 18, 6, 18, 6, 18, 18, 6, 18, 6]


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
from collections import Counter


class Vocabulary(object):
    """a vocabulary for NLP corpus"""

    def __init__(self, max_vocab_size=50000, min_count=None, start_end_tokens=False, name="Vocabulary1"):
        self.name = name
        self.max_vocab_size = max_vocab_size
        self.min_count = min_count
        self.start_end_tokens = start_end_tokens  # whether include start and end tokens
        self.vocabulary_size = 2
        self.word2idx = {"<UNK>": 0, "<PAD>": 1}
        self.idx2word = ["<UNK>", "<PAD>"]
        self.idx2count = [0, 0]

    def buildVocabulary(self, data):
        """
        method to build a vocabulary
        :param list data: a list of strings, with each string being a sentence
        :return: None
        """
        # add start and end token
        if self.start_end_tokens:
            self.idx2word += ['<SOS>', '<EOS>']
            self.vocabulary_size += 2

        # count words
        counter = Counter(
            [word for sentence in data for word in sentence.split()])
        # filter words by their counts
        if self.max_vocab_size:
            counter = {word: freq for word, freq in counter.most_common(self.max_vocab_size - self.vocabulary_size)}
        # filter words with low frequency
        if self.min_count:
            counter = {word: freq for word, freq in counter.items() if freq >= self.min_count}

        # generate attributes
        self.idx2word += list(sorted(counter.keys()))
        self.idx2count = [counter.get(word, 0) for word in self.idx2word]
        self.word2idx = {word: idx for idx, word in enumerate(self.idx2word)}
        self.vocabulary_size = len(self.idx2word)

    def indexer(self, word):
        """
        return word index
        :param str word: the word to query
        :return: word index
        """
        try:
            return self.word2idx[word]
        except KeyError:
            return self.word2idx['<UNK>']

In [ ]:
import jieba
import re
import random
import torch
import logging


# map Chinese punctuations into English punctuations
ch2en = {
    '！': '!',
    '？': '?',
    '。': '.',
    '（': '(',
    '）': ')',
    '，': ',',
    '：': ':',
    '；': ';',
    '｀': ','
}


def normalizeString(s):
    """
    lower case and trim punctuations and numbers
    :param str s: input string
    :return: normalised string
    """
    s = s.lower().strip()
    # check Chinese punctuations
    s = "".join(char if char not in ch2en.keys() else ch2en[char] for char in s)
    s = re.sub(r"([.!?])", r" \1", s)  # only keep . ! ?
    # keep alphabets, the three punctuations and Chinese characters
    s = re.sub(r"[^a-zA-Z.!?\u4e00-\u9fff]+", r" ", s)
    return s


def preprocess(data_path):
    """
    pre-process text data and return vocabularies
    :param str data_path: input data path
    :return: a list of lists of processed sentence pairs
    """
    # read data and split into pairs
    lines = open(data_path, encoding="utf-8").read().strip().split('\n')
    pairs = [[normalizeString(s) for s in line.split('\t')[:2]] for line in lines]

    # segment Chinese sentences
    # 这边会自己做分词
    for line in pairs:
        line[1] = " ".join(word for word in jieba.lcut(line[1]))

    return pairs

def Chpreprocess(data_path):
    """
    pre-process text data and return vocabularies
    :param str data_path: input data path
    :return: a list of lists of processed sentence pairs
    """
    # read data and split into pairs
    lines = open(data_path, encoding="utf-8").read().strip().split('\n')
    pairs = [[normalizeString(s) for s in line.split('\t')[:2]] for line in lines]

    # segment Chinese sentences
    # 这边会自己做分词
    for line in pairs:
        line[1] = " ".join(word for word in jieba.lcut(line[1]))
        line[0] = " ".join(word for word in jieba.lcut(line[0]))

    return pairs


def getVocabulary(pairs, input_lang, output_lang, max_vocab_size, reverse=False, start_end_tokens=True):
    """
    generate vocabularies for the pairs
    :param list pairs: language sentence pairs
    :param str input_lang: input language name
    :param str output_lang: output language name
    :param int max_vocab_size: max vocabulary size
    :param bool reverse: whether to inverse the input and output sentences
    :param bool start_end_tokens: whether to use start and end tokens
    :return: two vocabularies
    """
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]

    # initialise vocabularies
    input_vocab = Vocabulary(name=input_lang, start_end_tokens=start_end_tokens, max_vocab_size=max_vocab_size)
    output_vocab = Vocabulary(name=output_lang, start_end_tokens=start_end_tokens, max_vocab_size=max_vocab_size)
    input_sentences = []
    output_sentences = []

    # build vocabularies
    for pair in pairs:
        input_sentences.append(pair[0])
        output_sentences.append(pair[1])
    input_vocab.buildVocabulary(input_sentences)
    output_vocab.buildVocabulary(output_sentences)

    return input_vocab, output_vocab

def ChgetVocabulary(pairs, lang, max_vocab_size, reverse=False, start_end_tokens=True):
    """
    generate vocabularies for the pairs
    :param list pairs: language sentence pairs
    :param str input_lang: input language name
    :param str output_lang: output language name
    :param int max_vocab_size: max vocabulary size
    :param bool reverse: whether to inverse the input and output sentences
    :param bool start_end_tokens: whether to use start and end tokens
    :return: two vocabularies
    """
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]

    # initialise vocabularies
    vocab = Vocabulary(name=lang, start_end_tokens=start_end_tokens, max_vocab_size=max_vocab_size)
    sentences = []
    
    # build vocabularies
    for pair in pairs:
        sentences.append(pair[0])
        sentences.append(pair[1])
    vocab.buildVocabulary(sentences)

    return vocab


def sentence2index(vocabulary, sentence):
    """
    convert sentence into list of index
    :param Vocabulary vocabulary: the vocabulary of the language
    :param str sentence: input sentence
    :return: list of index
    """
    index = [vocabulary.indexer("<SOS>")]
    index += [vocabulary.indexer(word) for word in sentence.split()]
    index += [vocabulary.indexer("<EOS>")]
    return index


def train_test_split(data, test_split=0.3):
    """
    split data into train and text sets
    :param list data: text data list
    :param float test_split: text proportion
    :return: train data and test data
    """
    size = len(data)
    split = size - int(size * test_split)
    random.seed(66)
    random.shuffle(data)
    return data[:split], data[split:]


def translation(sentence, model, max_length, src_vocab, trg_vocab):
    """
    translate one source sentence
    :param str sentence: source sentence
    :param model: the Seq2Seq model
    :param int max_length: max sentence length
    :param src_vocab: source language vocabulary
    :param trg_vocab: target language vocabulary
    :return: one target sentence
    """
    sentence = normalizeString(sentence)
    index = sentence2index(src_vocab, sentence)
    source = torch.LongTensor(index).unsqueeze(1)  # (T, 1)
    model = model.cpu()
    target = model.translate(source, max_length)
    target = [trg_vocab.idx2word[idx] for idx in target]
    if target[-1] == "<EOS>":
        target = target[:-1]
    return "".join(word for word in target)


def create_logger(filename):
    """
    create a logger
    :param str filename: log file name include path
    :return: logger
    """
    logger = logging.getLogger(__name__)
    fmt = '%(message)s'
    format_str = logging.Formatter(fmt)  # formatting
    logger.setLevel(logging.INFO)  # level
    sh = logging.StreamHandler()  # output on screen
    sh.setFormatter(format_str)  
    th = logging.FileHandler(filename)
    th.setFormatter(format_str)  # output into file
    logger.addHandler(sh)  
    logger.addHandler(th)
    return logger

logger = create_logger("training.log")

'''

'\nif __name__ == "__main__":\n    data = preprocess(Config.data_path)\n    English, Chinese = getVocabulary(data, "English", "Chinese", 10000)\n    index = sentence2index(Chinese, data[2046][1])\n    print(data[2046][1])\n    print(index)\n    print([Chinese.idx2word[idx] for idx in index])'

In [ ]:
if __name__ == "__main__":
    data = Chpreprocess(Config.data_path)
    Chinese = ChgetVocabulary(data, "Chinese", 10000)
    index = sentence2index(Chinese, data[2046][1])
    print(data[2046][1])
    print(data[2046][0])
    print(index)
    print([Chinese.idx2word[idx] for idx in index])

郑州 在 哪个 省
郑州 是 哪个 省 的
[2, 9136, 2576, 2347, 6969, 3]
['<SOS>', '郑州', '在', '哪个', '省', '<EOS>']


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class TranslationData(Dataset):

    def __init__(self, data, src_vocab, trg_vocab):
        """
        Dataset for translation
        :param list data: processed data
        :param src_vocab: source language vocabulary
        :param trg_vocab: target language vocabulary
        """
        super(TranslationData, self).__init__()
        self.data = data
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_sentence = self.data[idx][0]
        trg_sentence = self.data[idx][1]
        src_index = sentence2index(self.src_vocab, src_sentence)
        trg_index = sentence2index(self.trg_vocab, trg_sentence)
        src = torch.LongTensor(src_index)
        trg = torch.LongTensor(trg_index)
        return src, trg


def collate_fn(batch):
    """
    customized collate function to dynamically padding mini-batches
    :param batch: the list of samples from Dataloader
    :return: padded mini-batch
    """
    pad_index = 1  # the <PAD> index in vocabulary
    src_list = [sample[0] for sample in batch]  # list of each language sentences
    trg_list = [sample[1] for sample in batch]

    def padding(sentence_list):
        """padding each sentence to the right"""
        max_len = max([sentence.size(0) for sentence in sentence_list])
        pad_sen = [sen.tolist() + [pad_index] * max(0, max_len - len(sen))
                   for sen in sentence_list]
        return torch.LongTensor(pad_sen).transpose(0, 1)  # shape of (T, B)

    return padding(src_list), padding(trg_list)


'if __name__ == "__main__":\n    data = Chpreprocess(Config.data_path)\n    src_vocab = ChgetVocabulary(data,"Chinese",10000)\n    trg_vocab = src_vocab\n    dataset = TranslationData(data, src_vocab, trg_vocab)\n    source, target = dataset[66]\n    print(source.size())\n    print(target.size())\n    loader = DataLoader(dataset, batch_size=4, collate_fn=collate_fn)\n    for i, batch in enumerate(loader):\n        source, target = batch\n        print(source.size())\n        print(target.size())\n        break'

In [ ]:
import torch
from torch.nn.utils import clip_grad_norm_
from datetime import datetime


def train(model, dataloader, optimizer, criterion, vocab_size, grad_clip, teacher_forcing):
    """
    training over one epoch
    :param model: the Seq2Seq model
    :param dataloader: training dataloader
    :param optimizer: training optimiser
    :param criterion: loss function
    :param vocab_size: target vocabulary size
    :param grad_clip: max gradient
    :param teacher_forcing: teacher forcing ratio for training
    :return: list of losses per 100 mini-batches
    """
    model.to(Config.device)
    model.train()
    batch_losses = []
    batch_loss = 0
    n_batches = len(dataloader)
    for i, batch in enumerate(dataloader):
        source, target = batch
        source = source.to(Config.device)
        target = target.to(Config.device)
        target_len = target.size(0)
        optimizer.zero_grad()
        output = model(source, target, teacher_forcing).to(Config.device)  # forward propagation
        #output = output.to(Config.device)
        loss = criterion(output[1:].view(-1, vocab_size),
                         target[1:].contiguous().view(-1))  # calculate NLL loss, ignore first token <SOS>
        loss.backward()  # backward propagation
        clip_grad_norm_(model.parameters(), grad_clip)  # clip gradients
        optimizer.step()  # update parameters
        batch_loss += loss.data.item() / target_len

        # print results every 100 mini-batches
        if i % 100 == 0 and i != 0:
            batch_loss = batch_loss / 100  # average loss
            batch_losses.append(batch_loss)
            logger.info("%s | Finished %.1f%% | Mini-batch %d | Avg Loss: %5.2f" %
                        (datetime.now().strftime('%H:%M:%S'), (i+1) / n_batches * 100, i+1, batch_loss))
            batch_loss = 0
    return batch_losses


def evaluate(model, dataloader, criterion, vocab_size):
    """
    evaluation over one epoch
    :param model: the Seq2Seq model
    :param dataloader: training dataloader
    :param criterion: loss function
    :param vocab_size: target vocabulary size
    :return: average evaluation loss
    """
    with torch.no_grad():
        model.eval()
        eval_loss = 0
        batch_loss = 0
        n_batches = len(dataloader)
        for i, batch in enumerate(dataloader):
            source, target = batch
            source = source.to(Config.device)
            target = target.to(Config.device)
            target_len = target.size(0)
            output = model(source, target, teacher_forcing=0.0)  # forward propagation
            output = output.to(Config.device)
            loss = criterion(output[1:].view(-1, vocab_size),
                             target[1:].contiguous().view(-1))  # calculate NLL loss
            eval_loss += loss.data.item() / target_len
            batch_loss += loss.data.item() / target_len

            # print results every 100 mini-batches
            if i % 100 == 0 and i != 0:
                batch_loss = batch_loss / 100  # average loss
                logger.info("%s | Finished %.1f%% | Mini-batch %d | Avg Loss: %5.2f" %
                            (datetime.now().strftime('%H:%M:%S'), (i + 1) / n_batches * 100, i + 1, batch_loss))
                batch_loss = 0
    return eval_loss / len(dataloader)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.nn import NLLLoss
from torch import optim

from datetime import datetime
import pickle

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.nn import NLLLoss
from torch import optim

from datetime import datetime
import pickle


'''# import and process data
data = preprocess(Config.data_path)
# data = data[:5000]  # test a small set
en_vocab, cn_vocab = getVocabulary(data, "English", "Chinese", Config.max_vocab_size)
train_data, eval_data = train_test_split(data, test_split=0.3)'''

# import and process data
data = Chpreprocess("/content/drive/MyDrive/all.txt")
#data = data[:5000]  # test a small set
cn_vocab = ChgetVocabulary(data, "Chinese", Config.max_vocab_size)
train_data, eval_data = train_test_split(data, test_split=0.3)

for parameter in Config.parameter_list[2:3]:
    print(parameter)
    embedding_size = parameter[0]
    hidden_size = parameter[1]
    dropout = parameter[2]
    teacher_forcing = parameter[3]
    batch_size = parameter[4]
    learning_rate = parameter[5]
    print("目前参数情况为:",embedding_size,',',hidden_size,',',dropout,',',teacher_forcing,',',batch_size,',',learning_rate)
    # get dataloader
    train_dataset = TranslationData(train_data, cn_vocab, cn_vocab)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)

    eval_dataset = TranslationData(eval_data, cn_vocab, cn_vocab)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=collate_fn)

    # prepare training
    encoder = Encoder(cn_vocab.vocabulary_size, embedding_size, hidden_size, Config.n_layers, dropout)
    decoder = Decoder(cn_vocab.vocabulary_size, embedding_size, hidden_size, Config.n_layers, dropout)
    model2 = Seq2Seq(encoder, decoder)
    model2 = model2.to(Config.device)

    optimizer = optim.Adam(model2.parameters(), lr=learning_rate)
    criterion = NLLLoss(ignore_index=1)  # ignore padding index

    # training epochs
    training_losses = []
    best_eval_loss = 100000
    for epoch in range(Config.epochs):
        logger.info(f"\n{datetime.now().strftime('%H:%M:%S')} | Training epoch {epoch+1} ...")
        logger.info("-" * 30)
        training_losses += train(model2, train_loader, optimizer, criterion, cn_vocab.vocabulary_size,
                  Config.grad_clip, teacher_forcing)
        logger.info(f"\n{datetime.now().strftime('%H:%M:%S')} | Evaluating epoch {epoch + 1} ...")
        logger.info("-" * 30)
        eval_loss = evaluate(model2, eval_loader, criterion, cn_vocab.vocabulary_size)
        print('eval_loss',eval_loss)
        if eval_loss < best_eval_loss:
            best_eval_loss = eval_loss
            logger.info(f"\n{datetime.now().strftime('%H:%M:%S')} | Saving model2 ...")
            torch.save(model2.state_dict(), "seq2seq.bin")
            with open('training_losses.bin', 'wb') as f:
                pickle.dump(training_losses, f)
        with open('eval_loss.bin', 'wb') as f:
            pickle.dump(eval_loss, f)
        sentence = "明天 要 出来 玩 吗"
        logger.info("\n[In] " + sentence)                  
        logger.info("[Out] " + translation(sentence, model2, Config.max_sentence_length, cn_vocab, cn_vocab))

        sentence = "脸大 适合 什么 发型"
        logger.info("\n[In] " + sentence)                  
        logger.info("[Out] " + translation(sentence, model2, Config.max_sentence_length, cn_vocab, cn_vocab))

        sentence = "如何 学 好 英语"
        logger.info("\n[In] " + sentence)                  
        logger.info("[Out] " + translation(sentence, model2, Config.max_sentence_length, cn_vocab, cn_vocab))


14:42:17 | Training epoch 1 ...

14:42:17 | Training epoch 1 ...

14:42:17 | Training epoch 1 ...

14:42:17 | Training epoch 1 ...
------------------------------
------------------------------
------------------------------
------------------------------


[300, 256, 0.5, 0.5, 32, 0.001]
目前参数情况为: 300 , 256 , 0.5 , 0.5 , 32 , 0.001


14:42:27 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.50
14:42:27 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.50
14:42:27 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.50
14:42:27 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.50
14:42:36 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.44
14:42:36 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.44
14:42:36 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.44
14:42:36 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.44
14:42:45 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.39
14:42:45 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.39
14:42:45 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.39
14:42:45 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.39
14:42:55 | Finished 42.3% | Mini-batch 401 | Avg Loss:  0.37
14:42:55 | Finished 42.3% | Mini-batch 401 | Avg Loss:  0.37
14:42:55 | Finished 42.3% | Mini-batch 401 | Avg Loss:  0.37
14:42:55 | Finished 42.3% | Mini-batch 401 | Avg Loss:  0.37
14:43:05 | Finished 52.8

eval_loss 0.282463613214085



[In] 脸大 适合 什么 发型

[In] 脸大 适合 什么 发型

[In] 脸大 适合 什么 发型

[In] 脸大 适合 什么 发型
[Out] 适合适合适合什么发型
[Out] 适合适合适合什么发型
[Out] 适合适合适合什么发型
[Out] 适合适合适合什么发型

[In] 如何 学 好 英语

[In] 如何 学 好 英语

[In] 如何 学 好 英语

[In] 如何 学 好 英语
[Out] 如何英语英语
[Out] 如何英语英语
[Out] 如何英语英语
[Out] 如何英语英语

14:43:57 | Training epoch 2 ...

14:43:57 | Training epoch 2 ...

14:43:57 | Training epoch 2 ...

14:43:57 | Training epoch 2 ...
------------------------------
------------------------------
------------------------------
------------------------------
14:44:07 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.26
14:44:07 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.26
14:44:07 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.26
14:44:07 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.26
14:44:16 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.25
14:44:16 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.25
14:44:16 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.25
14:44:16 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.25


...

eval_loss 0.5219150860632875


[Out] 如何学英语好
[Out] 如何学英语好
[Out] 如何学英语好
[Out] 如何学英语好

17:23:37 | Training epoch 99 ...

17:23:37 | Training epoch 99 ...

17:23:37 | Training epoch 99 ...

17:23:37 | Training epoch 99 ...
------------------------------
------------------------------
------------------------------
------------------------------
17:23:46 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.01
17:23:46 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.01
17:23:46 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.01
17:23:46 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.01
17:23:55 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.01
17:23:55 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.01
17:23:55 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.01
17:23:55 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.01
17:24:05 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.01
17:24:05 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.01
17:24:05 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.01
17:24:05 | Finish

eval_loss 0.521427924331078


[Out] 好的英语好
[Out] 好的英语好
[Out] 好的英语好
[Out] 好的英语好

17:25:16 | Training epoch 100 ...

17:25:16 | Training epoch 100 ...

17:25:16 | Training epoch 100 ...

17:25:16 | Training epoch 100 ...
------------------------------
------------------------------
------------------------------
------------------------------
17:25:26 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.01
17:25:26 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.01
17:25:26 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.01
17:25:26 | Finished 10.7% | Mini-batch 101 | Avg Loss:  0.01
17:25:35 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.01
17:25:35 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.01
17:25:35 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.01
17:25:35 | Finished 21.2% | Mini-batch 201 | Avg Loss:  0.01
17:25:45 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.01
17:25:45 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.01
17:25:45 | Finished 31.8% | Mini-batch 301 | Avg Loss:  0.01
17:25:45 | Finish

eval_loss 0.522228732979889


In [ ]:
model2

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10000, 300)
    (lstm): LSTM(300, 256, num_layers=2, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(10000, 300)
    (lstm): LSTM(556, 256, num_layers=2, dropout=0.5)
    (attention): Attention(
      (attn1): Linear(in_features=512, out_features=256, bias=True)
      (attn2): Linear(in_features=256, out_features=1, bias=True)
    )
    (out): Linear(in_features=512, out_features=10000, bias=True)
  )
)

In [ ]:
torch.save(model.state_dict(), "test22.bin")

In [ ]:
f=open('/content/drive/MyDrive/测试原句子.txt','r',encoding='utf-8')
test_lst=[]
lines = f.readlines()      #读取全部内容 ，并以列表方式返回
for line in lines:
    line=line.replace('\n','')
    line=line.replace('\t','')
    test_lst.append(line)

result2=[]
count=0
for word in test_lst:
  word = " ".join(word for word in jieba.lcut(word))
  resu = translation(word, model2, Config.max_sentence_length, cn_vocab, cn_vocab)
  print(word,' ',resu)
  temp=word+' '+resu
  result2.append(temp)
  count+=1
  if count % 100 == 0:
    f=open('result2.txt','w',encoding='utf-8')
    for line in result2:
      f.write(line+'\n')

流式输出内容被截断，只能显示最后 5000 行内容。
父亲 给 我 上 的 一堂课 阅读 答案   父亲给我上的<UNK>阅读答案
怎么 看 淘宝 年度 账单   淘宝年度看账单怎么
带字 女生 头像   女生头像头像
性价比 较 高 的 平板 电脑   是<UNK>平板电脑电脑
成龙 最新 电影 叫 什么   成龙最新电影电影什么
奶粉 什么 牌子 比较 好 呀   什么牌子的奶粉好
天涯 明月刀 电视剧 好看 吗 ？ 大家   天涯明月刀到底到底有人说他
虎落平阳 一 ( ) 歇后语   虎落平阳歇后语
手机 有 什么 软件 可以 赚钱   手机有什么软件可以赚钱
怎么 才能 去掉 黑痣   <UNK>怎么去掉
怎么 查找 同名同姓 的 人   如何查找同名同姓的人
脸上 的 麻子 怎么 去掉   脸上有麻子怎么去掉
镜子 和 照片 哪个 更 真实 些   照片和照片哪个更真实
迅雷 继续 下载 没 资源   迅雷下载资源不足
孙俪 和 邓超 离婚 吗   孙俪和邓超离婚了吗
油烟机 哪个 牌子 好   什么牌子油烟机和好
能 看 下 红包 的 使用 范围 么   什么红包的红包最好
我 的 世界 电脑 版 如何 下载   我的世界电脑版怎么下载啊
亵渎 是 什么 意思   <UNK>什么意思
补肾 该 吃 什么 食物 啊   吃什么食物补肾
如何 登陆 新浪 博客   怎么登陆新浪游戏
呵呵 你 一脸 什么 意思   呵呵你一脸什么意思
如何 查询 驾驶证 真假   如何查询驾驶证真假
怎样 使 大腿 变瘦   怎样让让历史变瘦
情人节 送 什么 礼物 给 男朋友   情人节送什么礼物給男朋友
我 怎么 才 可以 注册 谷歌 账户   怎么才可以注册自己谷歌账户
天天 飞车 怎样 开挂   咋下载天天飞车
魔兽 世界 为什么 进不去 啊   魔兽世界为什么进不去为什么
最 吝啬 的 人 打 一 成语   最吝啬的人猜
这是 怎样 的 漫画 风格 呢   这是什么漫画风格
财富 值 怎么 获取   怎样获取财富值
全民 飞机 大战 宠物 哪个 好   全民大战飞机宠物哪个好
什么 时候 天气 回暖 了   天气什么时候能回暖啊?冷死了
女士 手表 有 哪些 品牌   品牌女士手表有哪些
如何 才能 删除 淘宝 帐号   淘宝如何删除才能
穿越 火线 为什么 老是